In [23]:
import os
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

seed_number = 123

In [24]:
main_dir = os.path.abspath(os.getcwd())

main_dir

'/home/pedro/Downloads/xgboost'

### ETAPA 1: lendo a base de dados

In [25]:
train = pd.read_csv(f'{main_dir}/input/train.csv')
test = pd.read_csv(f'{main_dir}/input/test.csv')
train.shape, test.shape

((113754, 24), (28439, 24))

In [26]:
data = pd.concat([train,test],sort=True)
data.columns

Index(['Cloud3pm', 'Cloud9am', 'Date', 'Evaporation', 'Humidity3pm',
       'Humidity9am', 'Id', 'Location', 'MaxTemp', 'MinTemp', 'Pressure3pm',
       'Pressure9am', 'RainToday', 'RainTomorrow', 'Rainfall', 'Sunshine',
       'Temp3pm', 'Temp9am', 'WindDir3pm', 'WindDir9am', 'WindGustDir',
       'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'],
      dtype='object')

In [27]:
data.shape, data.columns

((142193, 24),
 Index(['Cloud3pm', 'Cloud9am', 'Date', 'Evaporation', 'Humidity3pm',
        'Humidity9am', 'Id', 'Location', 'MaxTemp', 'MinTemp', 'Pressure3pm',
        'Pressure9am', 'RainToday', 'RainTomorrow', 'Rainfall', 'Sunshine',
        'Temp3pm', 'Temp9am', 'WindDir3pm', 'WindDir9am', 'WindGustDir',
        'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'],
       dtype='object'))

In [20]:
data.head()

,Nuvem15h,Nuvem9h,Data,Evaporacao,Umidade15h,Umidade9h,ID,Localizacao,TempMaxima,TempMinima,...,Pluviosidade,Ensolarado,Temperatura15h,Temperatura9h,DirecaoVento15h,DirecaoVento9h,RajadadeVentoDirecao,VelocidadeRajadadeVento,VelocidadeVento15h,VelocidadeVento9h
0,NaN,NaN,2008-12-02,NaN,25.0,44.0,2,Albury,25.1,7.4,...,0.0,NaN,24.3,17.2,WSW,NNW,WNW,44.0,22.0,4.0
1,2.0,NaN,2008-12-03,NaN,30.0,38.0,3,Albury,25.7,12.9,...,0.0,NaN,23.2,21.0,WSW,W,WSW,46.0,26.0,19.0
2,NaN,NaN,2008-12-04,NaN,16.0,45.0,4,Albury,28.0,9.2,...,0.0,NaN,26.5,18.1,E,SE,NE,24.0,9.0,11.0
3,NaN,NaN,2008-12-06,NaN,23.0,55.0,6,Albury,29.7,14.6,...,0.2,NaN,28.9,20.6,W,W,WNW,56.0,24.0,19.0
4,NaN,NaN,2008-12-08,NaN,19.0,48.0,8,Albury,26.7,7.7,...,0.0,NaN,25.5,16.3,W,SSE,W,35.0,17.0,6.0


In [21]:
data.describe()

,Nuvem15h,Nuvem9h,Evaporacao,Umidade15h,Umidade9h,ID,TempMaxima,TempMinima,Pressao15h,Pressao9h,Pluviosidade,Ensolarado,Temperatura15h,Temperatura9h,VelocidadeRajadadeVento,VelocidadeVento15h,VelocidadeVento9h
count,85099.000000,88536.000000,81350.000000,138583.000000,140419.000000,142193.000000,141871.000000,141556.000000,128212.000000,128179.000000,140787.000000,74377.000000,139467.000000,141289.000000,132923.000000,139563.000000,140845.000000
mean,4.503167,4.437189,5.469824,51.482606,68.843810,72885.900649,23.226784,12.186400,1015.258204,1017.653758,2.349974,7.624853,21.687235,16.987509,39.984292,18.637576,14.001988
std,2.720633,2.887016,4.188537,20.797772,19.051293,42069.147355,7.117618,6.403283,7.036677,7.105476,8.465173,3.781525,6.937594,6.492838,13.588801,8.803345,8.893337
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-4.800000,-8.500000,977.100000,980.500000,0.000000,0.000000,-5.400000,-7.200000,6.000000,0.000000,0.000000
25%,2.000000,1.000000,2.600000,37.000000,57.000000,36236.000000,17.900000,7.600000,1010.400000,1012.900000,0.000000,4.900000,16.600000,12.300000,31.000000,13.000000,7.000000
50%,5.000000,5.000000,4.800000,52.000000,70.000000,73312.000000,22.600000,12.000000,1015.200000,1017.600000,0.000000,8.500000,21.100000,16.700000,39.000000,19.000000,13.000000
75%,7.000000,7.000000,7.400000,66.000000,83.000000,109248.000000,28.200000,16.800000,1020.000000,1022.400000,0.800000,10.600000,26.400000,21.600000,48.000000,24.000000,19.000000
max,9.000000,9.000000,145.000000,100.000000,100.000000,145459.000000,48.100000,33.900000,1039.600000,1041.000000,371.000000,14.500000,46.700000,40.200000,135.000000,87.000000,130.000000


In [22]:
data.describe(include=['O'])

,Data,Localizacao,ChoveHoje,ChoveAmanha,DirecaoVento15h,DirecaoVento9h,RajadadeVentoDirecao
count,142193,142193,140787,142193,138415,132180,132863
unique,3436,49,2,2,16,16,16
top,2017-05-10,Canberra,No,No,SE,N,W
freq,49,3418,109332,110316,10663,11393,9780


In [28]:
# checando quais atributos são numéricos (não consegui exibir o dataframe inteiro no vscode)
numerics =  data.select_dtypes(include = np.number).columns.to_list()
print(numerics)

"""
17 atributos numéricos no total
"""

['Cloud3pm', 'Cloud9am', 'Evaporation', 'Humidity3pm', 'Humidity9am', 'Id', 'MaxTemp', 'MinTemp', 'Pressure3pm', 'Pressure9am', 'Rainfall', 'Sunshine', 'Temp3pm', 'Temp9am', 'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am']


In [30]:
#checando atributos categoricos
categorical = data.select_dtypes('object').columns.to_list()
print(categorical)

"""
7 atributos categóricos
"""

['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindDir3pm', 'WindDir9am', 'WindGustDir']


In [ ]:
# o ID é inútil pq só atribui um valor unico pra a instancia do dado então vou dropar
data.drop("Id", axis=1, inplace=True)
data